In [ ]:
from .population import Population

class PopulationElitist(Population):

    def __init__(self):
        super().__init__()
        self.fronts = []

    def fastNondominatedSort(self):
        G = []
        for r in self.individuals:
            r.domination_count = 0
            r.dominated_solutions = []
            for q in self.individuals:
                if r.dominates(q):
                    r.dominated_solutions.append(q)
                elif q.dominates(r):
                    r.domination_count += 1
            if r.domination_count == 0:
                G.append(r)
                r.rank = 1
        self.fronts.append(G)
        k = 1
        while(len(self.fronts[k-1]) > 0):
            G = []
            for r in self.fronts[k-1]:
                for q in r.dominated_solutions:
                    q.domination_count -= 1
                    if q.domination_count == 0:
                        G.append(q)
                        q.rank = k + 1
            self.fronts.append(G)
            k += 1
        if len(self.fronts[-1]) == 0:
            del self.fronts[-1]
        
    def crowdingDistanceSortFront(self, k):
        front = self.fronts[k]
        L = len(front) - 1
        for i in range(front[0].problem.m):
            front.sort(key = lambda individual : individual.objectives[i])
            front[0].crowding_distance = float('inf')
            front[L].crowding_distance = float('inf')
            normalization_constant = front[L].objectives[i] - front[0].objectives[i]
            if normalization_constant == 0:
                normalization_constant = 1
            for j in range(1,L-1):
                front[j].crowding_distance += (front[j+1].objectives[i] - front[j-1].objectives[i]) / normalization_constant
        front.sort(key = lambda individual : individual.crowding_distance, reverse = True)

    def crowdingDistanceSort(self):
        for k in range(len(self.fronts)):
            self.crowdingDistanceSortFront(k)

    def elitist_selection(self, population, N):
        k = 0
        while (self.N + len(population.fronts[k]) < N):
            self.append_many(population.fronts[k])
            k += 1
        if self.N < N:
            population.crowdingDistanceSortFront(k)
            self.append_many(population.fronts[k][0:N-self.N])

    def summary(self):
        print("Population size: ", self.N)
        k = 1
        for front in self.fronts:
            print("Front:", k, "\tIndividuals:", len(front))
            k += 1

    
        





